<H2> Project -6 English to French Translator </H2>


In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Embedding, RepeatVector, TimeDistributed
import numpy as np

from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint


ModuleNotFoundError: No module named 'keras'

<H3> Loading both English and French files</H3>

In [2]:
eng_sent = open('small_vocab_en.txt', encoding='utf-8').read().split('\n')
fra_sent = open('small_vocab_fr.txt', encoding='utf-8').read().split('\n')

<H4> Length of English and French file </H4>

In [3]:
len(eng_sent)

137861

In [4]:
len(fra_sent)

137861

<H3> Get unique number of characters in both English and French files </H3>

In [5]:
eng_chars = []
fra_chars = []

In [6]:
nb_samples = len(fra_sent)

In [7]:
nb_samples

137861

In [8]:
for line in range(nb_samples):
    eng_line = eng_sent[line]
    fra_line = fra_sent[line]
    for ch in eng_line:
        if (ch not in eng_chars):
            eng_chars.append(ch)
    for ch in fra_line:
        if (ch not in fra_chars):
            fra_chars.append(ch)

done


In [ ]:
# repition of the for loop
for line in range(nb_samples):
    
        eng_line = eng_sent[line]    
        fra_line = fra_sent[line]
    
    for ch in eng_line:
        if (ch not in eng_chars):
            eng_chars.append(ch)
            
    for ch in fra_line:
        if (ch not in fra_chars):
            fra_chars.append(ch)

In [ ]:
31  - unique english characters
40 - unique french characters

In [9]:
len(fra_chars)  # unique french characters

40

In [10]:
len(eng_chars) # unique English characters

31

In [11]:
fra_sent[0]

"new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."

In [12]:
eng_sent[0]

'new jersey is sometimes quiet during autumn , and it is snowy in april .'

<H3> Create a Dictionary where it will map every index to a english character and a reverse dictionary to map every english  character to a index </H3>

In [13]:
# dictionary to index each english character - key is index and value is english character
eng_index_to_char_dict = {}
# dictionary to get english character given its index - key is english character and value is index
eng_char_to_index_dict = {}

for k, v in enumerate(eng_chars):
    eng_index_to_char_dict[k] = v
    eng_char_to_index_dict[v] = k

<H3> Create a Dictionary where it will map every index to a french character and a reverse dictionary to map every french character to a index </H3> 

In [14]:
# dictionary to index each french character - key is index and value is french character
fra_index_to_char_dict = {}

# dictionary to get french character given its index - key is french character and value is index
fra_char_to_index_dict = {}

for k, v in enumerate(fra_chars):
    fra_index_to_char_dict[k] = v
    fra_char_to_index_dict[v] = k


In [ ]:
# Length of each  maximum sentences in both english and french and get the maximum length

In [15]:
max_len_eng_sent = max([len(line) for line in eng_sent])
max_len_fra_sent = max([len(line) for line in fra_sent])

In [16]:
# Sort the characters
fra_chars = sorted(fra_chars)
eng_chars = sorted(eng_chars)

In [18]:
print(max_len_eng_sent)
print(max_len_fra_sent)

102
114


In [19]:
tokenized_eng_sentences = np.zeros(shape = (nb_samples,max_len_eng_sent,len(eng_chars)), dtype='float32')
tokenized_fra_sentences = np.zeros(shape = (nb_samples,max_len_fra_sent,len(fra_chars)), dtype='float32')
target_data = np.zeros((nb_samples, max_len_fra_sent, len(fra_chars)),dtype='float32')

<H5> Replace characters with the corresponding numbers from the dictionary and create a 3 dimensional vectors </H5>

In [20]:
# Vectorize the english and french sentences

for i in range(nb_samples):
    for k,ch in enumerate(eng_sent[i]):
        tokenized_eng_sentences[i,k,eng_char_to_index_dict[ch]] = 1
        
    for k,ch in enumerate(fra_sent[i]):
        tokenized_fra_sentences[i,k,fra_char_to_index_dict[ch]] = 1

        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i,k-1,fra_char_to_index_dict[ch]] = 1


In [21]:
# Encoder model

encoder_input = Input(shape=(None,len(eng_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]


In [22]:
# Decoder model

decoder_input = Input(shape=(None,len(fra_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(fra_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)


<H4>Model creation and running and training  </H4>

In [ ]:

model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x=[tokenized_eng_sentences,tokenized_fra_sentences], 
          y=target_data,
          batch_size=64,
          epochs=50,
          validation_split=0.2)


Train on 110288 samples, validate on 27573 samples
Epoch 1/50
110288/110288 [==============================] - 188680s 2s/step - loss: 0.3800 - val_loss: 0.1430
Epoch 2/50
 30912/110288 [=======>......................] - ETA: 1:36:25 - loss: 0.1233

In [ ]:
# Inference models for testing

# Encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )


In [ ]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(fra_chars)))
    #target_seq[0, 0, fra_char_to_index_dict['\t']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    while not stop_condition:
        
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_fra_char = fra_index_to_char_dict[max_val_index]
        translated_sent += sampled_fra_char
        
        if ( (sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(fra_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
        
    return translated_sent


In [ ]:
#my_sent = ["I love french fries", "He is an English man"]
my_sent = ["I love french fries"]


In [ ]:
tokenized_eng_input = np.zeros(shape = (1,max_len_eng_sent,len(eng_chars)), dtype='float32')
for i in range(1):
    for k,ch in enumerate(eng_sent[i]):
        tokenized_eng_input[i,k,eng_char_to_index_dict[ch]] = 1
decode_seq(tokenized_eng_input)


In [ ]:
for seq_index in range(10):
    inp_seq = tokenized_eng_sentences[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-')
    print('Input sentence:', eng_sent[seq_index])
    print('Decoded sentence:', translated_sent)
